In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 24 12:49:04 2018

@author: Jimmy
"""

import pandas as pd
import numpy as np
import requests
import re
import glob
#from gensim.models.word2vec import Word2Vec
import os

import jieba
import jieba.analyse
from hanziconv import HanziConv

from functools import reduce
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

jieba_path = r'C:\Users\Jimmy\Anaconda3\Lib\site-packages\jieba'

#load jieba dictionary
jieba.set_dictionary(os.path.join(jieba_path, 'dict.txt'))

list_url = glob.glob(r'C:\Users\Jimmy\Desktop\Python\Googleresult\CCCC\1?.html')
#list_url = [url1,url2,url3]

def read_from_file(list_url):
    google_df = pd.DataFrame(columns = ['heading', 'content', 'google_tag', 'extract_tag'])
    for url in list_url:
        with open(url, 'rb') as f:
            soup = BeautifulSoup(f.read().decode('utf8'), features = "html.parser")
        for articles in soup.find_all('div', class_='g'):
            if not re.search(re.compile('youtube\.com|img|books.google.com'), articles.prettify()):
                heading = articles.find('a').text
                content = articles.find('span', class_='st').text
                #replace date and  ...
                cleanr = re.compile(u'[0-9]{1,2}[ ]?月[0-9]{1,2}[ ]?日|\.{3}|[.]{3}|\\xa0|On\.cc|i-Cable|China Press|\.\.')
                heading = re.sub(cleanr, '', heading)
                content = re.sub(cleanr, '', content)
                google_tag = ''
                for tag in articles.find('div', class_='s').find_all('b')[1:]:
                    if tag != r'...':
                        google_tag = google_tag + ' ' + tag.text
                row = pd.Series([heading,content,google_tag,'0'], index = ['heading', 'content', 'google_tag', 'extract_tag'])
                google_df = google_df.append(row,ignore_index=True)
    return google_df

#creating a DF contain heading, content, extract tag,

def google_content(list_url):
    google_df = pd.DataFrame(columns = ['heading', 'content', 'google_tag', 'extract_tag'])
    for url in list_url:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "lxml")
        for articles in soup.find_all('div', class_='g'):
            if not re.search(re.compile('youtube\.com|img|books.google.com'),articles.prettify()):
                heading = articles.find('a').text
                content = articles.find('span', class_='st').text
                #replace date and  ...
                cleanr = re.compile(u'[0-9]{1,2}[ ]?月[0-9]{1,2}[ ]?日|\.{3}|[.]{3}|\\xa0|On\.cc|i-Cable|China Press|\.\.')
                heading = re.sub(cleanr, '', heading)
                content = re.sub(cleanr, '', content)
                google_tag = ''
                for tag in articles.find('div', class_='s').find_all('b')[1:]:
                    if tag != r'...':
                        google_tag = google_tag + ' ' + tag.text
                row = pd.Series([heading,content,google_tag,'0'], index = ['heading', 'content', 'google_tag', 'extract_tag'])
                google_df = google_df.append(row,ignore_index=True)
    return google_df

# text cutting by jieba, convert to traditional word at the same time
    
def sim_to_trad(df):
    df[['heading', 'content']] = df[['heading','content']].applymap(lambda x : HanziConv.toSimplified(x))
    
def cleaning(df):
    cleanr1 = re.compile(u'[0-9]{1,2}月[0-9]{1,2}日|\.{3}|[0-9] hours ago')
    cleanr2 = re.compile(u'[0-9]{1} day[s]? ago|[0-9]{1,2} hour[s]? ago|[0-9]{1,2} min[s]? ago')
    df[['heading', 'content']] = df[['heading','content']].applymap(lambda x: re.sub(cleanr1, '', x))
    df[['heading', 'content']] = df[['heading','content']].applymap(lambda x: re.sub(cleanr2, 'recent', x))
#    aka = re.compile(u'中國交建|中交建|中交|中國交通建設集團有限公司|中國交通建設')
#    df[['heading','content']] = df[['heading','content']].applymap(lambda x: re.sub(aka, '中交', x))

def extract_tag(df):
    for index in df.index:
        cut_heading = jieba.analyse.extract_tags(HanziConv.toSimplified(df.loc[index,'heading']), topK =50)
        cut_content = jieba.analyse.extract_tags(HanziConv.toSimplified(df.loc[index,'content']), topK =50)
        df.loc[index,'extract_tag'] = reduce(lambda x,y: x + ' ' + y,cut_heading + cut_content)
        df.loc[index,'len_cut_heading'] = len(cut_heading)
        df.loc[index,'len_cut_content'] = len(cut_content)
        #print(df.loc[index,'extract_tag'])

def tfidf(df):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(df['extract_tag'])

def tfidf_2(df):
    vectorizer = TfidfVectorizer()
    return vectorizer.fit_transform(df['extract_tag'])

def svd(tfidf):
    svd = TruncatedSVD(n_components=int(tfidf.shape[1]/25), n_iter=7, random_state=42)
    matrix = svd.fit_transform(tfidf)
    print(svd.explained_variance_ratio_.sum())  
    return matrix

def nmf(tfidf):
    model = NMF(n_components=int(60), random_state=42, max_iter=20)
    matrix = model.fit_transform(tfidf)
    return matrix

def KM_clustering_selection(df, matrix):
    KM = KMeans(n_jobs=-1)
    KM = KMeans(random_state=42)
    classifier = GridSearchCV(KM,
                                    {"n_clusters": np.logspace(np.log10(df.shape[0] / 100),
                                                               np.log10(df.shape[0] / 1.5), 5).astype(int)},
                                    n_jobs=-1)
    classifier.fit(matrix)
    df['labels'] = classifier.best_estimator_.labels_
    df['distance_point_to_cluster'] = classifier.best_estimator_.transform(matrix).min(axis=1)
    print("inertia {}".format(classifier.best_estimator_.inertia_))
    print("Number of Cluster {}".format(df['labels'].nunique()))
       
    
def printing_result(df):
    print(df['labels'].nunique())
    for label in df['labels'].value_counts().index[:20]:
        print('cluster {}'.format(label))
        tag = ''
        for index in df[df['labels'] == label].index:
            tag = tag + df.loc[index,'heading'] + df.loc[index,'content']
        print(google_df[google_df['labels'] == label]['distance_point_to_cluster'].mean())
        print(jieba.analyse.extract_tags(HanziConv.toSimplified(tag),topK =10))
        print(df['content'][df['labels'] == label])


google_df = read_from_file(list_url)
sim_to_trad(google_df)
cleaning(google_df)
extract_tag(google_df)
#google_tfidf = tfidf(google_df)
google_tfidf = tfidf_2(google_df)
#matrix = svd(google_tfidf)
matrix = nmf(google_tfidf)
KM_clustering_selection(google_df,matrix)
printing_result(google_df)

Building prefix dict from C:\Users\Jimmy\Anaconda3\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\Jimmy\AppData\Local\Temp\jieba.u68658d17e004c479affaffef87308a74.cache
Loading model cost 1.004 seconds.
Prefix dict has been built succesfully.


inertia 0.2171675336250949
Number of Cluster 620
620
cluster 199
0.032472194298112066
['中交建', '中交', '招股价', '造岛', '2015', '4.3%', '586', '亿建', '上返', '孙政才']
169    2014年 - 五粮液(000858-CN)在周二披露的《投资者关系活动记录表》中表示，白酒...
289    2016年 - 真相网／程义）随着首相纳吉访华之行带来丰硕成果，在野党近日来频频发放不实消息...
299    2015年 - 欧盟警告特朗普:全球将对3000亿美元美产品报复 · 去东莞！华为1500辆...
535     - 记者12日从应急管理部了解到，统计数据显示，今年以来全国建筑业事故总量同比上升4.3%...
543    精准施策，打赢蓝天保卫战 · 经过三年努力，将大幅减少主要大气污染物排放总量 【阅读】 · ...
684    中国港湾是香港上市的中交股份公司的全资子公司，代表中交股份在国际工程  美国政府今天投票重新...
722    黄玉治赶赴“5.29”透水事故现场 · 586亿建经济圈实现公路免费深圳通走遍深莞惠 586...
822    2015年 - 陆勇：慢性粒细胞白血病患者、英雄与商人 · 为了房事，曾国藩花掉一年工资 ·...
890    2007年 - 你不需要太过担心! 一定可以上返招股价! 一旦上返招股价, 建议沽出. 可以...
894    2018年 - 财新记者向中交建、中铁建相关负责人求证此事，他们均表示不想再多谈  案的审查...
900    2017年 - 北京100多万农户到这个采暖季能够用上电和燃气来替代取暖，比过去烧煤干净安全...
901    2017年 - 作为巫家坝首次大规模出让的土地，此次竞拍却意外的平淡，4个打包地块  今年，...
910    中交建遗憾东铁暂停工．“实质工作进展良好”  总检察长汤米汤姆斯表示，对于前首相拿督斯里纳吉...
919    福建中交建新型环保建材有限公司怎么样? 安徽休宁县公益用地建别墅奢华装修实景求曝光在公益用地...
920    1 - 实际上美国拿所谓「商业间谍」、「侵犯智慧财产权」、「偷窃技术」等  在西

0.005581472969514007
['中望', 'WWW', 'ZWCAD', '中交', '三局', '龙腾', '企业形象', '侥幸心理', '知名企业', '股份']
71     如果抱有侥幸心理，一旦侵权纠纷曝光，企业形象将受到严重损害，甚至还会带来  招商地产、国投集...
100    如果抱有侥幸心理，一旦侵权纠纷曝光，企业形象将受到严重损害，甚至还会带来  招商地产、国投集...
643    如果抱有侥幸心理，一旦侵权纠纷曝光，企业形象将受到严重损害，甚至还会带来  招商地产、国投集...
Name: content, dtype: object
cluster 54
0.031750224412179555
['中交', '游戏', '2018', '平台', '手机', '民工', '赌博', '里程', '论文', '双块']
59     要说游戏大平台【w83kk.com】,玩真钱的手机赌博游戏是最有实力的之一,【适合玩游戏的手...
691    2018年 - 据悉，以上几位四川民工曾在台江县台盘乡中交股份沪昆客专贵州段工程第三项目务工...
701    2018年 - 中交股份新建云桂铁路（广西段）YGZQ-2标段无砟轨道起点里程DK124+8...
Name: content, dtype: object
cluster 225
0.01531505966097632
['交通', '有限公司', '建设', '股份', '国务院', '中交', '2006', '百科', '中国', '国资委']
356    中国交通建设股份有限公司（以下简称中交股份）成立于2006年，经国务院批准，由中国交通建设集...
448    中国交通建设股份有限公司（以下简称中交股份）成立于2006年，经国务院批准，由中国交通建设集...
636    2018年 - 中国交通建设股份有限公司（以下简称中交股份）成立于2006年，经国务院批准，...
Name: content, dtype: object
cluster 82
0.027515056533200962
['中交建', '东海岸', '马来西亚', '大马', '项目', '发声明', '表示遗憾', '2018', '铁路', '工程']
213    2018年

In [23]:
list_index = list(google_df['labels'].value_counts().isin([1]).index)

In [26]:
google_df[google_df['labels'].isin(list_index)]['content']

0      2018年 - 根据《中华人民共和国审计法》的规定，2017年5月至6月，审计署对中国  至...
1      拟募资200亿元，或为A股年内最大IPO项目的中国交通建设股份有限公司（以下简称中交股份），...
2      2011年 - 中国网讯中国交通建设股份有限公司（以下称中交股份）因被指孙公司有染“豆腐渣”...
3      2018年 - 6.2010年至审计时，所属中交股份19名职工投资入股的公司违规与集团内部企...
4      6．2010年至审计时，所属中交股份19名职工投资入股的公司违规与集团内部企业发生业务往来1...
5      2011年 - 中交股份因被指孙公司有染“豆腐渣”公路工程建设，并涉嫌刻意隐瞒全资子公司环境...
6      2011年 - 网民对中交股份涉嫌违法上市的行为表达了不满。根据证券法规定，股份有限公司申请...
7      2011年 - 中交股份回应污染万亩海洋曾向证监会专题汇报. 编者按：中交股份“环保门”事件...
8      和讯股票消息，经证监会发审委第216次会议审核，中国交通建设股份有限公司首次公开发行A股申请...
9      中交股份：回应高价收购F&G质疑, ，香港上市公司中国交通建设（行情,资讯,评论）股份有限公...
10     2010年 - 中国证监会上周五晚间发布公告称，发审委定于召开2011年第216次工作会议，...
11     中国交通建设股份有限公司（以下简称中交股份）成立于2006年，经国务院  然而也有很多人提出...
12     证券时报记者获悉，，经证监会发审委第216次会议审核，中国交通建设股份有限公司首次公开发行A...
13     中交股份”本次A股IPO拟发行不超过35亿股，发行后总股本不超过183.25亿股。中交股份过...
14     2018年 - 俄罗斯卫星通讯社北京电新仰光发展公司（NYDC）与中国交通建设股份有限公司一...
15     2011年 - 证券时报记者获悉，，经证监会发审委第216次会议审核，中国交通建设股份有限公...
16     2011年 - 有学者认为，一旦监管部门因乳山港工程施工污染事件对中交烟台公司予以处罚，必将...
17     2011年 - 虽然在中交股份的招股书和官网中都